In [1]:
from sympy import symbols

## Define Input Parameters

In [2]:
# define camera parameters:
fx, fy, cx, cy = symbols('fx fy cx cy')

In [3]:
# select intermediate variables as landmark position in camera frame:
P_X, P_Y, P_Z = symbols('P_X P_Y P_Z')

## Point-to-Pixel Projection

In [4]:
# perspective division:
p_x = P_X/P_Z
p_y = P_Y/P_Z

In [5]:
# conversion to pixel coordinates:
p_x_prime = fx * p_x + cx
p_y_prime = fy * p_y + cy

## Derivative

In [6]:
from sympy import Matrix, simplify, diff

In [7]:
# define Jacobian with respect to intermediate variables:
JP = Matrix(
    [
        [diff(p_x_prime, P_X), diff(p_x_prime, P_Y), diff(p_x_prime, P_Z)],
        [diff(p_y_prime, P_X), diff(p_y_prime, P_Y), diff(p_y_prime, P_Z)]
    ]
)

In [8]:
# jacobian with respect to camera parameters:
JI = Matrix(
    [
        [diff(p_x_prime, fx), diff(p_x_prime, fy), diff(p_x_prime, cx), diff(p_x_prime, cy)],
        [diff(p_y_prime, fx), diff(p_y_prime, fy), diff(p_y_prime, cx), diff(p_y_prime, cy)]
    ]    
)

In [9]:
# Jacobian of Lie algebra:
TP = Matrix(
    [
        [1, 0, 0,    0,  P_Z, -P_Y],
        [0, 1, 0, -P_Z,    0,  P_X],
        [0, 0, 1,  P_Y, -P_X,    0]
    ]
)

In [10]:
# overall Jacobians:

# a. Jacobian for landmark position:
J_position = simplify(JP)
# b. Jacobian for camera intrinsics:
J_intrinsic = simplify(JI)
# c. Jacobian for camera pose:
J_pose = simplify(JP * TP)

## Show Result

In [11]:
from sympy import init_printing
init_printing()

In [12]:
# display Jacobian for camera pose:
J_position

⎡ fx       -P_X⋅fx ⎤
⎢───   0   ────────⎥
⎢P_Z            2  ⎥
⎢            P_Z   ⎥
⎢                  ⎥
⎢      fy  -P_Y⋅fy ⎥
⎢ 0   ───  ────────⎥
⎢     P_Z       2  ⎥
⎣            P_Z   ⎦

In [13]:
# display Jacobian for camera intrinsics:
J_intrinsic

⎡P_X           ⎤
⎢───   0   1  0⎥
⎢P_Z           ⎥
⎢              ⎥
⎢     P_Y      ⎥
⎢ 0   ───  0  1⎥
⎣     P_Z      ⎦

In [14]:
# display Jacobian for landmark position:
J_pose

⎡                                       2                  ⎤
⎢ fx       -P_X⋅fx    -P_X⋅P_Y⋅fx    P_X ⋅fx       -P_Y⋅fx ⎥
⎢───   0   ────────   ────────────   ─────── + fx  ────────⎥
⎢P_Z            2            2            2          P_Z   ⎥
⎢            P_Z          P_Z          P_Z                 ⎥
⎢                                                          ⎥
⎢                         2                                ⎥
⎢      fy  -P_Y⋅fy     P_Y ⋅fy        P_X⋅P_Y⋅fy    P_X⋅fy ⎥
⎢ 0   ───  ────────  - ─────── - fy   ──────────    ────── ⎥
⎢     P_Z       2           2               2        P_Z   ⎥
⎣            P_Z         P_Z             P_Z               ⎦

## Statements for C++ Eigen

In [15]:
def format_jacobian_as_cpp_eigen(J, variable, col_offset):
    """
    Format SymPy Jacobian as CPP Eigen expression
    """    
    def format_element(J, i, j):
        """
        Format Jacobian matrix element
        """
        import re
        
        # get element representation:
        element_expression = repr(simplify(J[i, j]))
        # find all element exponentials:
        element_exponentials = {e for e in re.findall(r'P_([X|Y|Z])\*{2}(\d)', element_expression)}
        
        # format all element exponentials
        for element_exponential in element_exponentials:
            c, e = element_exponential
            element_expression = re.sub(f'P_{c}\*\*{e}', f'P_{c}_{e}', element_expression)
        
        return element_exponentials, f'{variable}({i}, {j+col_offset}) = {element_expression};'
    
    # get matrix dimension:
    M, N = J.shape
    
    # init variables and assignments:
    variables = set()
    assignments = []
    
    # generate:
    for m in range(M):
        for n in range(N):
            # get variables and assignment:
            vs, a = format_element(J, m, n)
            variables = variables.union(vs)
            assignments.append(a)
    
    # format as C++ statement:
    print("// this is the analytic Jacobian expression generated by SymPy")
    # a. declarations:
    for c, e in sorted(variables, key=lambda x: f'{x[0]}{x[1]}'):
        print(f'double P_{c}_{e} = pow({c}, {e});')
    # b. Jacobian:
    print(f'\nEigen::Matrix<double, {M}, {N}> {variable};\n')
    for assignment in assignments:
        print(assignment)
    
    return None

In [16]:
# show jacobian for intermediate variables:
format_jacobian_as_cpp_eigen(J_position, 'JP', 0)

// this is the analytic Jacobian expression generated by SymPy
double P_Z_2 = pow(Z, 2);

Eigen::Matrix<double, 2, 3> JP;

JP(0, 0) = fx/P_Z;
JP(0, 1) = 0;
JP(0, 2) = -P_X*fx/P_Z_2;
JP(1, 0) = 0;
JP(1, 1) = fy/P_Z;
JP(1, 2) = -P_Y*fy/P_Z_2;


In [17]:
# show jacobian for camera intrinsics:
format_jacobian_as_cpp_eigen(J_intrinsic, 'JI', 0)

// this is the analytic Jacobian expression generated by SymPy

Eigen::Matrix<double, 2, 4> JI;

JI(0, 0) = P_X/P_Z;
JI(0, 1) = 0;
JI(0, 2) = 1;
JI(0, 3) = 0;
JI(1, 0) = 0;
JI(1, 1) = P_Y/P_Z;
JI(1, 2) = 0;
JI(1, 3) = 1;


In [18]:
# show jacobian for camera pose:
format_jacobian_as_cpp_eigen(J_pose, '_jacobianOplusXi', 0)

// this is the analytic Jacobian expression generated by SymPy
double P_X_2 = pow(X, 2);
double P_Y_2 = pow(Y, 2);
double P_Z_2 = pow(Z, 2);

Eigen::Matrix<double, 2, 6> _jacobianOplusXi;

_jacobianOplusXi(0, 0) = fx/P_Z;
_jacobianOplusXi(0, 1) = 0;
_jacobianOplusXi(0, 2) = -P_X*fx/P_Z_2;
_jacobianOplusXi(0, 3) = -P_X*P_Y*fx/P_Z_2;
_jacobianOplusXi(0, 4) = P_X_2*fx/P_Z_2 + fx;
_jacobianOplusXi(0, 5) = -P_Y*fx/P_Z;
_jacobianOplusXi(1, 0) = 0;
_jacobianOplusXi(1, 1) = fy/P_Z;
_jacobianOplusXi(1, 2) = -P_Y*fy/P_Z_2;
_jacobianOplusXi(1, 3) = -P_Y_2*fy/P_Z_2 - fy;
_jacobianOplusXi(1, 4) = P_X*P_Y*fy/P_Z_2;
_jacobianOplusXi(1, 5) = P_X*fy/P_Z;
